In [2]:
import os
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import loadmat

import einops
import tensorflow as tf
from tensorflow import einsum
from tensorflow.image import ssim as tf_ssim, psnr as tf_psnr

import keras
import keras.backend as K
from keras import Model
from keras import layers
from keras.layers import *
from keras.optimizers import AdamW
from keras.losses import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
from keras.callbacks import ReduceLROnPlateau

def check_tf_gpu():
    sys_info = tf.sysconfig.get_build_info()
    version, cuda, cudnn = tf.__version__, sys_info["cuda_version"], sys_info["cudnn_version"]
    count = len(tf.config.experimental.list_physical_devices())
    name  = [device.name for device in tf.config.experimental.list_physical_devices('GPU')]
    print('-'*66)
    print('-'*26, 'VERSION INFO', '-'*26)
    print('TF version: {} | # Device(s) available: {}'.format(version, count))
    print('TF Built with CUDA? {} | CUDA: {} | cuDNN: {}'.format(tf.test.is_built_with_cuda(), cuda, cudnn))
    print(tf.config.list_physical_devices()[0],'\n', tf.config.list_physical_devices()[1])
    print('-'*66+'\n')
    return None

check_tf_gpu()

------------------------------------------------------------------
-------------------------- VERSION INFO --------------------------
TF version: 2.15.0 | # Device(s) available: 2
TF Built with CUDA? True | CUDA: 12.0 | cuDNN: 8
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU') 
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
------------------------------------------------------------------



In [ ]:
sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

n_timesteps = 33
nx, ny, nz, nz_short  = 100, 100, 11, 5

indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['gci']
Grid = np.zeros((nx,ny,nz)).flatten(order='F')
Grid[indexMap] = 1
Grid = Grid.reshape(nx,ny,nz, order='F')
Tops = np.load('data_npy_100_100_11/tops_grid.npz')['tops']
print('Grid: {} | Tops: {}'.format(Grid.shape, Tops.shape))

In [ ]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension. 
    """
    super().__init__()
    self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [ ]:
def make_model():
    K.clear_session()
    inp_xm = Input(shape=(29128, 2))
    inp_xg = Input(shape=(29128, 2))
    inp_xw = Input(shape=(2, 5))
    inp_xc = Input(shape=(None, 5))
    inp_xt = Input(shape=(None, 1))
    inputs = [inp_xm, inp_xg, inp_xw, inp_xc, inp_xt]

    def conv_block(inp, filt):
        _ = Masking(mask_value=0.)(inp)
        _ = Conv2Plus1D(filters=filt, kernel_size=(3,3,3), padding='same')(_)
        _ = GroupNormalization(groups=-1)(_)
        _ = Activation('gelu')(_)
        _ = Conv2Plus1D(filters=filt, kernel_size=(3,3,3), padding='same')(_)
        _ = GroupNormalization(groups=-1)(_)
        _ = Activation('gelu')(_)
        _ = MaxPooling3D((2,2,1))(_)
        return _
    
    def triple_dense_block(inp, units):
        def dense_block(inp, units):
            _ = Dense(units)(inp)
            _ = GroupNormalization(groups=-1)(_)
            _ = Activation('gelu')(_)
            return _
        _ = dense_block(inp, units)
        _ = dense_block(_, units)
        _ = dense_block(_, units)
        return _
    
    def recc_block(inp, units):
        _ = LSTM(units, return_sequences=True)(inp)
        _ = LayerNormalization()(_)
        _ = Activation('gelu')(_)
        _ = LSTM(units, return_sequences=True)(_)
        _ = LayerNormalization()(_)
        _ = Activation('gelu')(_)
        return _
    
    def decon_block(inp, filt):
        _ = TimeDistributed(Conv2Plus1D(filters=filt, kernel_size=(3,3,3), padding='same'))(inp)
        _ = GroupNormalization(groups=-1)(_)
        _ = Activation('gelu')(_)
        _ = TimeDistributed(Conv2Plus1D(filters=filt, kernel_size=(3,3,3), padding='same'))(_)
        _ = GroupNormalization(groups=-1)(_)
        _ = Activation('gelu')(_)
        _ = TimeDistributed(Conv3DTranspose(filt//4, 3, strides=(2,2,1), padding='same'))(_)
        return _
    
    # xm = conv_block(inp_xm, 64)
    # xm = conv_block(xm, 128)

    # xg = conv_block(inp_xg, 64)
    # xg = conv_block(xg, 128)

    xm = triple_dense_block(inp_xm, 100)
    xg = triple_dense_block(inp_xg, 100)

    xw = recc_block(inp_xw, 50)
    xw = triple_dense_block(xw, 100)

    xc = recc_block(inp_xc, 50)
    xc = triple_dense_block(xc, 100)

    xt = recc_block(inp_xt, 50)
    xt = triple_dense_block(xt, 100)

    mg = einsum('bpc, bpc -> bpc', xm, xg)
    wc = einsum('blc, btc -> btlc', xw, xc)

    branch = einsum('bpc, btlc -> btpc', mg, wc)
    trunk  = einsum('btpc, btc -> btpc', branch, xt)

    x = Dense(2)(trunk)
    

    # x = decon_block(trunk, 128)
    # x = decon_block(x, 64)
    # x = TimeDistributed(Conv2Plus1D(filters=2, kernel_size=(3,3,3), padding='same'))(x)

    return Model(inputs=inputs, outputs=x)

In [ ]:
def custom_loss(true, pred, alpha=0.66, beta=0.33):
    ssim_loss = tf.reduce_mean(1.0 - tf_ssim(true, pred, max_val=1.0))
    mse_loss  = MeanSquaredError()(true, pred)
    mae_loss  = MeanAbsoluteError()(true, pred)
    ridge_loss = beta*mae_loss + (1-beta)*mse_loss
    return alpha*ssim_loss + (1-alpha)*ridge_loss

In [ ]:
model = make_model()
print('# Parameters: {:,}'.format(model.count_params()))

In [ ]:
train_idx = np.random.choice(range(1272), size=500, replace=False)
test_idx  = np.setdiff1d(range(1272), train_idx)

In [ ]:
# xm = np.zeros((len(train_idx), nx, ny, nz_short, 2))
# xg = np.zeros((len(train_idx), nx, ny, nz_short, 2))
xm = np.zeros((len(train_idx), 29128, 2))
xg = np.zeros((len(train_idx), 29128, 2))
xw = np.zeros((len(train_idx), 2, 5))
xc = np.zeros((len(train_idx), n_timesteps, 5))
xt = np.zeros((len(train_idx), n_timesteps, 1))
yy = np.zeros((len(train_idx), 33, 29128, 2))
Grid_ext = np.repeat(np.expand_dims(Grid, 0), 33, axis=0)

def apply_mask(x, imap, mask_value=0.0):
    xx = mask_value*np.ones((nx,ny,nz)).flatten(order='F')
    xx[imap] = x.flatten(order='F')[imap]
    xx = xx.reshape((nx,ny,nz), order='F')[...,5:10]
    return xx

for i in range(len(train_idx)):
    # xg[i] = np.expand_dims(np.concatenate([np.expand_dims(Grid[...,5:10], -1), np.expand_dims(Tops[...,5:10], -1)], -1), 0)
    g = np.expand_dims(Grid.flatten(order='F')[indexMap], -1)
    t = np.expand_dims(Tops.flatten(order='F')[indexMap], -1)
    xg[i] = np.expand_dims(np.concatenate([g, t], -1), 0)

    m = np.load('data_npy_100_100_11/inputs_rock_rates_locs_time/x_{}.npz'.format(train_idx[i]))
    # p = np.expand_dims(apply_mask(m['poro'], indexMap), -1)
    # k = np.expand_dims(apply_mask(m['perm'], indexMap, mask_value=-9), -1)
    p = np.expand_dims(m['poro'].flatten(order='F')[indexMap], -1)
    k = np.expand_dims(m['perm'].flatten(order='F')[indexMap], -1)
    xm[i] = np.concatenate([p, k], -1)

    xw[i] = m['locs']
    xc[i] = m['ctrl']
    xt[i] = m['time']

    dd = np.load('data_npy_100_100_11/outputs_pressure_saturation/y_{}.npz'.format(train_idx[i]))
    prm = dd['pressure'].reshape(33, -1, order='F')[:,indexMap]
    sam = dd['saturation'].reshape(33, -1, order='F')[:,indexMap]
    yy[i,...,0] = prm
    yy[i,...,1] = sam

print('xm', xm.shape)
print('xg', xg.shape)
print('xw', xw.shape)
print('xc', xc.shape)
print('xt', xt.shape)
print('yy', yy.shape)

In [ ]:
num_epochs = 100
batch_size = 32

In [ ]:
class LossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 10 == 0:
            print('Epoch: [{}/{}] - Loss: {:.4f} | Val Loss: {:.4f}'.format(epoch+1, num_epochs, logs['loss'], logs['val_loss']))

In [ ]:
model.compile(optimizer=AdamW(learning_rate=1e-3, weight_decay=4e-3), loss=custom_loss, metrics=['mse'])
start = time()
history = model.fit(x=[xm, xg, xw, xc, xt], y=yy, 
                    batch_size       = batch_size, 
                    epochs           = num_epochs, 
                    validation_split = 0.2, 
                    verbose          = False)
train_time = time() - start

***
# END